In [1]:
pip install fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import pandas as pd
from scipy.sparse import hstack, csr_matrix

In [2]:
# Paso 3: Cargar el modelo entrenado y los codificadores
model = joblib.load('crime_prediction_model.pkl')
onehot_encoder = joblib.load('onehot_encoder.pkl')

In [3]:
# Paso 4: Definir los modelos de datos para la entrada y salida de la API
class CrimePredictionInput(BaseModel):
    DATE_OF_OCCURRENCE: str
    BLOCK: str
    IUCR: str
    LOCATION_DESCRIPTION: str
    ARREST: str
    DOMESTIC: str
    BEAT: int
    WARD: int

class CrimePredictionOutput(BaseModel):
    PRIMARY_DESCRIPTION: str

In [4]:
# Paso 5: Inicializar la aplicación FastAPI
app = FastAPI()

In [5]:
# Paso 6: Definir el endpoint para hacer predicciones de crimen
@app.post('/predict/')
async def predict_crime(data: CrimePredictionInput):
    # Convertir los datos de entrada en un DataFrame
    input_data = pd.DataFrame([data.dict()])
    
    # Convertir las fechas al formato adecuado
    input_data['Date'] = pd.to_datetime(input_data['Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

    # Extraer características relevantes de las fechas
    input_data['YEAR'] = input_data['Date'].dt.year
    input_data['MONTH'] = input_data['Date'].dt.month
    input_data['DAY'] = input_data['Date'].dt.day
    input_data['HOUR'] = input_data['Date'].dt.hour

    # Eliminar la columna original de fecha
    input_data = input_data.drop(columns=['Date'])
    
    # Codificar las variables categóricas con OneHotEncoder
    input_data_encoded = onehot_encoder.transform(input_data.select_dtypes(include=['object']))

    # Combinar las características codificadas con las numéricas
    input_data_numerical = input_data.select_dtypes(exclude=['object']).astype(float)
    input_data_numerical = csr_matrix(input_data_numerical.values)  # Convertir a matriz dispersa
    input_data_combined = hstack((input_data_encoded, input_data_numerical))

    # Hacer la predicción
    prediction = model.predict(input_data_combined)

    # Obtener la descripción primaria del crimen predicho
    primary_description = prediction[0]

    return CrimePredictionOutput(PRIMARY_DESCRIPTION=primary_description)

In [6]:
# Ejecutar la aplicación con Uvicorn en modo de recarga
if __name__ == '__main__':
    import uvicorn
    uvicorn.run(app, host='0.0.0.0', port=8000, reload=True)

SystemExit: 1

/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
